# EDA 
1. General plots of demographics -> breakdown by republican and democrats?
2. Maybe some plots on which county switched from D -> R or R-> D
3. Definitely some choloropleth plots for both 2012, 2016 results and 2020 predictions
4. Get updated demographics for 2020?

Classes needed for EDA? Unlikely
Functions? Probably - plotting, chloropleth plots with county data
Function for calculating electoral college votes. pd.merge on state?
One hot encoder to prep